In [2]:
import pandas as pd
import numpy as np
import folium
import os
from folium.plugins import HeatMap

In [3]:
years = range(2022,2026)
df_all = pd.concat(
    [pd.read_csv(f'../data/raw/feminicide_{year}.csv') for year in years],
    ignore_index=True
)

In [4]:
df_all = df_all.rename(columns=lambda x: x.strip().lower().replace(' ', '_'))

In [5]:
df_all.columns = (
    df_all.columns
    .str.normalize("NFKD")
    .str.encode("ascii", "ignore")
    .str.decode("utf-8")
)

In [6]:

#renomme les colonnes et suppression des colonnes inutiles
df_all = df_all.rename(columns={'quand':'date','lieu':'commune'})
df_all.drop(columns=['n','contenu_publication','lien_instagram'], inplace=True)
df_all.head()


,date,commune,departement,prenom,age
0,25/11/2022,Prahecq,Deux-Sèvres,Leslie,22.0
1,04/01/2023,Perles,Aisne,–,50.0
2,31/12/2022,Charleville-Mézières,Ardennes,–,23.0
3,24/12/2022,Saint-Raphaël,Var,Molka,31.0
4,16/10/2022,Goudelin,Côtes-d'Armor,Jacqueline,67.0


In [7]:
#modification du format de la date
df_all['date'] = pd.to_datetime(df_all['date'], format='%d/%m/%Y', errors='coerce')
df_all.head()

,date,commune,departement,prenom,age
0,2022-11-25,Prahecq,Deux-Sèvres,Leslie,22.0
1,2023-01-04,Perles,Aisne,–,50.0
2,2022-12-31,Charleville-Mézières,Ardennes,–,23.0
3,2022-12-24,Saint-Raphaël,Var,Molka,31.0
4,2022-10-16,Goudelin,Côtes-d'Armor,Jacqueline,67.0


In [8]:
#créer commune normalisée (sans accent, minuscule, avec tiret à la place des espaces et des apostrophes)
df_all['nom_sans_accent'] = (
    df_all['commune']
    .str.normalize("NFKD")
    .str.encode("ascii", "ignore")
    .str.decode("utf-8")
    .str.lower()
    .str.replace(" ", "-", regex=False)
    .str.replace("'", "-", regex=False)
)

df_all['dep_sans_accent'] = (
    df_all['departement']
    .str.normalize("NFKD")
    .str.encode("ascii", "ignore")
    .str.decode("utf-8")
    .str.lower()
    .str.replace(" ", "-", regex=False)
    .str.replace("'", "-", regex=False)
)


In [9]:
#suppression des espaces et tiret au début et à la fin de nom_sans_accent et dep_sans_accent
df_all['nom_sans_accent'] = df_all['nom_sans_accent'].str.strip('- ') 
df_all['dep_sans_accent'] = df_all['dep_sans_accent'].str.strip('- ') 

df_all['nom_sans_accent'] = df_all['nom_sans_accent'].str.rstrip('- ') 
df_all['dep_sans_accent'] = df_all['dep_sans_accent'].str.rstrip('- ') 

df_all.head(100)

,date,commune,departement,prenom,age,nom_sans_accent,dep_sans_accent
0,2022-11-25,Prahecq,Deux-Sèvres,Leslie,22.0,prahecq,deux-sevres
1,2023-01-04,Perles,Aisne,–,50.0,perles,aisne
2,2022-12-31,Charleville-Mézières,Ardennes,–,23.0,charleville-mezieres,ardennes
3,2022-12-24,Saint-Raphaël,Var,Molka,31.0,saint-raphael,var
4,2022-10-16,Goudelin,Côtes-d'Armor,Jacqueline,67.0,goudelin,cotes-d-armor
...,...,...,...,...,...,...,...
95,2022-05-16,Béziers,Hérault,Claire,38.0,beziers,herault
96,2022-05-09,Saint-Rémy,Saône-et-Loire,Sarah,32.0,saint-remy,saone-et-loire
97,2022-05-09,Gergy,Saône-et-Loire,Audrey,33.0,gergy,saone-et-loire
98,2022-05-08,Grézieu-la-Varenne,Rhône,Nathalie,33.0,grezieu-la-varenne,rhone


In [10]:
#dans df_all, remplacer les noms de commune contenant '*e arrondissement' par 'Paris'
df_all['nom_sans_accent'] = df_all['nom_sans_accent'].str.replace(r'\d{1,2}e arrondissement', 'paris', regex=True)
df_all.head(100)

,date,commune,departement,prenom,age,nom_sans_accent,dep_sans_accent
0,2022-11-25,Prahecq,Deux-Sèvres,Leslie,22.0,prahecq,deux-sevres
1,2023-01-04,Perles,Aisne,–,50.0,perles,aisne
2,2022-12-31,Charleville-Mézières,Ardennes,–,23.0,charleville-mezieres,ardennes
3,2022-12-24,Saint-Raphaël,Var,Molka,31.0,saint-raphael,var
4,2022-10-16,Goudelin,Côtes-d'Armor,Jacqueline,67.0,goudelin,cotes-d-armor
...,...,...,...,...,...,...,...
95,2022-05-16,Béziers,Hérault,Claire,38.0,beziers,herault
96,2022-05-09,Saint-Rémy,Saône-et-Loire,Sarah,32.0,saint-remy,saone-et-loire
97,2022-05-09,Gergy,Saône-et-Loire,Audrey,33.0,gergy,saone-et-loire
98,2022-05-08,Grézieu-la-Varenne,Rhône,Nathalie,33.0,grezieu-la-varenne,rhone


In [11]:

df_all['nom_sans_accent'] = df_all['nom_sans_accent'].str.replace('bourgoin-jallieu-/-lyon', 'bourgoin-jallieu')

#remplcer dep_sans_accent de rhone à isere pour bourgoin-jallieu
df_all.loc[df_all['nom_sans_accent']=='bourgoin-jallieu', 'dep_sans_accent'] = 'isere'


In [12]:
df_commune = pd.read_csv('../data/raw/communes-france-2025.csv')
df_commune.head()

C:\Users\ghirg\AppData\Local\Temp\ipykernel_9572\954040633.py:1: DtypeWarning: Columns (1,12,14,16,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df_commune = pd.read_csv('../data/raw/communes-france-2025.csv')


,Unnamed: 0,code_insee,nom_standard,nom_sans_pronom,nom_a,nom_de,nom_sans_accent,nom_standard_majuscule,typecom,typecom_texte,...,longitude_mairie,latitude_centre,longitude_centre,grille_densite,grille_densite_texte,niveau_equipements_services,niveau_equipements_services_texte,gentile,url_wikipedia,url_villedereve
0,0,01001,L'Abergement-Clémenciat,Abergement-Clémenciat,à Abergement-Clémenciat,de l'Abergement-Clémenciat,l-abergement-clemenciat,L'ABERGEMENT-CLÉMENCIAT,COM,commune,...,4.921,46.153,4.926,6,Rural à habitat dispersé,0.0,communes non pôle,NaN,https://fr.wikipedia.org/wiki/fr:L'Abergement-...,https://villedereve.fr/ville/01001-l-abergemen...
1,1,01002,L'Abergement-de-Varey,Abergement-de-Varey,à Abergement-de-Varey,de l'Abergement-de-Varey,l-abergement-de-varey,L'ABERGEMENT-DE-VAREY,COM,commune,...,5.423,46.009,5.428,6,Rural à habitat dispersé,0.0,communes non pôle,"Abergementais, Abergementaises",https://fr.wikipedia.org/wiki/fr:L'Abergement-...,https://villedereve.fr/ville/01002-l-abergemen...
2,2,01004,Ambérieu-en-Bugey,Ambérieu-en-Bugey,à Ambérieu-en-Bugey,d'Ambérieu-en-Bugey,amberieu-en-bugey,AMBÉRIEU-EN-BUGEY,COM,commune,...,5.360,45.961,5.373,2,Centres urbains intermédiaires,3.0,centres structurants d'équipements et de services,"Ambarrois, Ambarroises",https://fr.wikipedia.org/wiki/fr:Ambérieu-en-B...,https://villedereve.fr/ville/01004-amberieu-en...
3,3,01005,Ambérieux-en-Dombes,Ambérieux-en-Dombes,à Ambérieux-en-Dombes,d'Ambérieux-en-Dombes,amberieux-en-dombes,AMBÉRIEUX-EN-DOMBES,COM,commune,...,4.903,45.996,4.912,5,Bourgs ruraux,1.0,centres locaux d'équipements et de services,Ambarrois,https://fr.wikipedia.org/wiki/fr:Ambérieux-en-...,https://villedereve.fr/ville/01005-amberieux-e...
4,4,01006,Ambléon,Ambléon,à Ambléon,d'Ambléon,ambleon,AMBLÉON,COM,commune,...,5.601,45.750,5.594,6,Rural à habitat dispersé,0.0,communes non pôle,Ambléonais,https://fr.wikipedia.org/wiki/fr:Ambléon,https://villedereve.fr/ville/01006-ambleon


In [13]:
# dans df_commune, récupérer commmune contenant 'Paris'
df_commune = df_commune[['nom_sans_accent','dep_nom','dep_code','population','superficie_km2','densite','latitude_mairie','longitude_mairie']]

In [14]:
df_commune.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34935 entries, 0 to 34934
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   nom_sans_accent   34935 non-null  object 
 1   dep_nom           34935 non-null  object 
 2   dep_code          34935 non-null  object 
 3   population        34935 non-null  int64  
 4   superficie_km2    34935 non-null  int64  
 5   densite           34932 non-null  float64
 6   latitude_mairie   34935 non-null  float64
 7   longitude_mairie  34935 non-null  float64
dtypes: float64(3), int64(2), object(3)
memory usage: 2.1+ MB


In [15]:
#créer commune normalisée (sans accent, minuscule, avec tiret à la place des espaces et des apostrophes)
df_commune['nom_sans_accent'] = (
    df_commune['nom_sans_accent']
    .str.normalize("NFKD")
    .str.encode("ascii", "ignore")
    .str.decode("utf-8")
    .str.lower()
    .str.replace(" ", "-", regex=False)
    .str.replace("'", "-", regex=False)
)

df_commune['dep_sans_accent'] = (
    df_commune['dep_nom']
    .str.normalize("NFKD")
    .str.encode("ascii", "ignore")
    .str.decode("utf-8")
    .str.lower()
    .str.replace(" ", "-", regex=False)
    .str.replace("'", "-", regex=False)
)

In [16]:
df_commune.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34935 entries, 0 to 34934
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   nom_sans_accent   34935 non-null  object 
 1   dep_nom           34935 non-null  object 
 2   dep_code          34935 non-null  object 
 3   population        34935 non-null  int64  
 4   superficie_km2    34935 non-null  int64  
 5   densite           34932 non-null  float64
 6   latitude_mairie   34935 non-null  float64
 7   longitude_mairie  34935 non-null  float64
 8   dep_sans_accent   34935 non-null  object 
dtypes: float64(3), int64(2), object(4)
memory usage: 2.4+ MB


In [17]:
#inner join entre df_all et df_commune sur les colonnes 'commune_sans_accents' et 'departement_sans_accents' de df_all et 'nom_sans_accent' et 'dep_sans_accents' de df_commune
df_merged = pd.merge(df_all, 
                     df_commune, 
                     how='left', 
                     left_on=['nom_sans_accent','dep_sans_accent'], 
                     right_on=['nom_sans_accent','dep_sans_accent'])

In [18]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 529 entries, 0 to 528
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   date              529 non-null    datetime64[ns]
 1   commune           529 non-null    object        
 2   departement       529 non-null    object        
 3   prenom            528 non-null    object        
 4   age               527 non-null    float64       
 5   nom_sans_accent   529 non-null    object        
 6   dep_sans_accent   529 non-null    object        
 7   dep_nom           453 non-null    object        
 8   dep_code          453 non-null    object        
 9   population        453 non-null    float64       
 10  superficie_km2    453 non-null    float64       
 11  densite           453 non-null    float64       
 12  latitude_mairie   453 non-null    float64       
 13  longitude_mairie  453 non-null    float64       
dtypes: datetime64[ns](1), floa

In [19]:
df_merged.head(100)

,date,commune,departement,prenom,age,nom_sans_accent,dep_sans_accent,dep_nom,dep_code,population,superficie_km2,densite,latitude_mairie,longitude_mairie
0,2022-11-25,Prahecq,Deux-Sèvres,Leslie,22.0,prahecq,deux-sevres,Deux-Sèvres,79,2246.0,25.0,90.0,46.259,-0.345
1,2023-01-04,Perles,Aisne,–,50.0,perles,aisne,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-12-31,Charleville-Mézières,Ardennes,–,23.0,charleville-mezieres,ardennes,Ardennes,08,46398.0,32.0,1460.0,49.760,4.720
3,2022-12-24,Saint-Raphaël,Var,Molka,31.0,saint-raphael,var,Var,83,35950.0,91.0,395.0,43.425,6.768
4,2022-10-16,Goudelin,Côtes-d'Armor,Jacqueline,67.0,goudelin,cotes-d-armor,Côtes-d'Armor,22,1720.0,23.0,74.0,48.605,-3.016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2022-05-16,Béziers,Hérault,Claire,38.0,beziers,herault,Hérault,34,80341.0,96.0,840.0,43.343,3.214
96,2022-05-09,Saint-Rémy,Saône-et-Loire,Sarah,32.0,saint-remy,saone-et-loire,Saône-et-Loire,71,6541.0,10.0,636.0,46.770,4.828
97,2022-05-09,Gergy,Saône-et-Loire,Audrey,33.0,gergy,saone-et-loire,Saône-et-Loire,71,2582.0,39.0,66.0,46.876,4.947
98,2022-05-08,Grézieu-la-Varenne,Rhône,Nathalie,33.0,grezieu-la-varenne,rhone,Rhône,69,6094.0,7.0,814.0,45.748,4.694


In [20]:
df_merged.rename(columns={'latitude_mairie': 'latitude', 'longitude_mairie': 'longitude'}, inplace=True)

In [21]:
 #enlever les lignes où com_norm est NaN
df_merged_sans_nan = df_merged[~df_merged['dep_code'].isna()]
df_merged_sans_nan.info()

<class 'pandas.core.frame.DataFrame'>
Index: 453 entries, 0 to 527
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   date             453 non-null    datetime64[ns]
 1   commune          453 non-null    object        
 2   departement      453 non-null    object        
 3   prenom           452 non-null    object        
 4   age              452 non-null    float64       
 5   nom_sans_accent  453 non-null    object        
 6   dep_sans_accent  453 non-null    object        
 7   dep_nom          453 non-null    object        
 8   dep_code         453 non-null    object        
 9   population       453 non-null    float64       
 10  superficie_km2   453 non-null    float64       
 11  densite          453 non-null    float64       
 12  latitude         453 non-null    float64       
 13  longitude        453 non-null    float64       
dtypes: datetime64[ns](1), float64(6), object(7)
mem

In [22]:
df_merged_nan = df_merged[df_merged['dep_code'].isna()]
df_merged_nan.info()

<class 'pandas.core.frame.DataFrame'>
Index: 76 entries, 1 to 528
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   date             76 non-null     datetime64[ns]
 1   commune          76 non-null     object        
 2   departement      76 non-null     object        
 3   prenom           76 non-null     object        
 4   age              75 non-null     float64       
 5   nom_sans_accent  76 non-null     object        
 6   dep_sans_accent  76 non-null     object        
 7   dep_nom          0 non-null      object        
 8   dep_code         0 non-null      object        
 9   population       0 non-null      float64       
 10  superficie_km2   0 non-null      float64       
 11  densite          0 non-null      float64       
 12  latitude         0 non-null      float64       
 13  longitude        0 non-null      float64       
dtypes: datetime64[ns](1), float64(6), object(7)
memo

In [23]:
import time
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="geoapi_commune")

def get_coords(commune):
    query = f"{commune} "
    try:
        location = geolocator.geocode(query, timeout=5)  # délai plus long
        if location:
            return location.latitude, location.longitude
    except Exception as e:
        print(f"Erreur pour {query}: {e}")
    return None, None

# Pour stocker les résultats déjà trouvés
cache = {}

for commune in df_merged_nan['commune']:
    key = commune
    if key not in cache:  # pas encore cherché
        lat, lon = get_coords(commune)
        cache[key] = (lat, lon)
        time.sleep(1)  # respecter la limite Nominatim
    else:
        lat, lon = cache[key]
    
    df_merged_nan.loc[
        (df_merged_nan['commune'] == commune),
        ['latitude', 'longitude']
    ] = [lat, lon]

In [ ]:
idx_non_nan = df_merged_nan[~df_merged_nan['latitude'].isna()].index
df_merged_sans_nan = pd.concat([df_merged_sans_nan, df_merged_nan.loc[idx_non_nan]], ignore_index=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 595 entries, 0 to 594
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   date             595 non-null    datetime64[ns]
 1   commune          595 non-null    object        
 2   departement      595 non-null    object        
 3   prenom           594 non-null    object        
 4   age              592 non-null    float64       
 5   nom_sans_accent  595 non-null    object        
 6   dep_sans_accent  595 non-null    object        
 7   dep_nom          453 non-null    object        
 8   dep_code         453 non-null    object        
 9   population       453 non-null    float64       
 10  superficie_km2   453 non-null    float64       
 11  densite          453 non-null    float64       
 12  latitude         595 non-null    float64       
 13  longitude        595 non-null    float64       
dtypes: datetime64[ns](1), float64(6), object(7

In [ ]:
df_merged_sans_nan.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 595 entries, 0 to 594
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   date             595 non-null    datetime64[ns]
 1   commune          595 non-null    object        
 2   departement      595 non-null    object        
 3   prenom           594 non-null    object        
 4   age              592 non-null    float64       
 5   nom_sans_accent  595 non-null    object        
 6   dep_sans_accent  595 non-null    object        
 7   dep_nom          453 non-null    object        
 8   dep_code         453 non-null    object        
 9   population       453 non-null    float64       
 10  superficie_km2   453 non-null    float64       
 11  densite          453 non-null    float64       
 12  latitude         595 non-null    float64       
 13  longitude        595 non-null    float64       
dtypes: datetime64[ns](1), float64(6), object(7

In [28]:
df_merged_nan = df_merged_nan[df_merged_nan['latitude'].isna()]
df_merged_nan

,date,commune,departement,prenom,age,nom_sans_accent,dep_sans_accent,dep_nom,dep_code,population,superficie_km2,densite,latitude,longitude
129,2022-02-19,Amnéville-les-Thermes,Moselle,Jocelyne,65.0,amneville-les-thermes,moselle,NaN,NaN,NaN,NaN,NaN,NaN,NaN
184,2023-10-01,Sainte-Maure-de-Peyrac,Lot-et-Garonne,Séverine,47.0,sainte-maure-de-peyrac,lot-et-garonne,NaN,NaN,NaN,NaN,NaN,NaN,NaN
378,2024-04-05,Equilivaz/La Salle,Autre ou hors France,Auriane,22.0,equilivaz/la-salle,autre-ou-hors-france,NaN,NaN,NaN,NaN,NaN,NaN,NaN
423,2024-01-01,Boucaya,Nouvelle-Calédonie,Lucie,29.0,boucaya,nouvelle-caledonie,NaN,NaN,NaN,NaN,NaN,NaN,NaN
445,2025-07-07,Coulange-lès-Nevers,Nièvre,Simone,90.0,coulange-les-nevers,nievre,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
#correction manuelle des noms de communes_sans_accent mal orthographiés
df_merged_nan.loc[(df_merged_nan['nom_sans_accent']=='amneville-les-thermes'),'nom_sans_accent'] = 'amneville'
df_merged_nan.loc[(df_merged_nan['nom_sans_accent']=='sainte-maure-de-peyrac'),'nom_sans_accent'] = 'sainte-maure-de-peyriac'
df_merged_nan.loc[(df_merged_nan['nom_sans_accent']=='equilivaz/la-salle'),'nom_sans_accent'] = 'la-salle-italia'
df_merged_nan.loc[(df_merged_nan['nom_sans_accent']=='boucaya'),'nom_sans_accent'] = 'bacouya'
df_merged_nan.loc[(df_merged_nan['nom_sans_accent']=='coulange-les-nevers'),'nom_sans_accent'] = 'coulanges-les-nevers'

#corection des communes mal orthographiées
df_merged_nan.loc[(df_merged_nan['commune']=='Amnéville-les-Thermes'),'commune'] = 'Amnéville'
df_merged_nan.loc[(df_merged_nan['commune']=='Sainte-Maure-de-Peyrac'),'commune'] = 'Sainte-Maure-de-Peyriac'
df_merged_nan.loc[(df_merged_nan['commune']=='Equilivaz/La Salle'),'commune'] = 'La Salle-Italia'
df_merged_nan.loc[(df_merged_nan['commune']=='Boucaya'),'commune'] = 'Bacouya'
df_merged_nan.loc[(df_merged_nan['commune']=='Coulange-lès-Nevers'),'commune'] = 'Coulanges-lès-Nevers'


In [31]:
df_merged_nan.head()

,date,commune,departement,prenom,age,nom_sans_accent,dep_sans_accent,dep_nom,dep_code,population,superficie_km2,densite,latitude,longitude
129,2022-02-19,Amnéville-les-Thermes,Moselle,Jocelyne,65.0,amneville,moselle,NaN,NaN,NaN,NaN,NaN,NaN,NaN
184,2023-10-01,Sainte-Maure-de-Peyriac,Lot-et-Garonne,Séverine,47.0,sainte-maure-de-peyriac,lot-et-garonne,NaN,NaN,NaN,NaN,NaN,NaN,NaN
378,2024-04-05,La Salle-Italia,Autre ou hors France,Auriane,22.0,la-salle-italia,autre-ou-hors-france,NaN,NaN,NaN,NaN,NaN,NaN,NaN
423,2024-01-01,Bacouya,Nouvelle-Calédonie,Lucie,29.0,bacouya,nouvelle-caledonie,NaN,NaN,NaN,NaN,NaN,NaN,NaN
445,2025-07-07,Coulanges-lès-Nevers,Nièvre,Simone,90.0,coulanges-les-nevers,nievre,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
cache = {}
for commune in df_merged_nan['commune']:
    key = commune
    if key not in cache:  # pas encore cherché
        lat, lon = get_coords(commune)
        cache[key] = (lat, lon)
        time.sleep(1)  # respecter la limite Nominatim
    else:
        lat, lon = cache[key]
    
    df_merged_nan.loc[
        (df_merged_nan['commune'] == commune),
        ['latitude', 'longitude']
    ] = [lat, lon]

In [36]:
df_merged_nan.head()

,date,commune,departement,prenom,age,nom_sans_accent,dep_sans_accent,dep_nom,dep_code,population,superficie_km2,densite,latitude,longitude
129,2022-02-19,Amnéville,Moselle,Jocelyne,65.0,amneville,moselle,NaN,NaN,NaN,NaN,NaN,49.260361,6.142074
184,2023-10-01,Sainte-Maure-de-Peyriac,Lot-et-Garonne,Séverine,47.0,sainte-maure-de-peyriac,lot-et-garonne,NaN,NaN,NaN,NaN,NaN,44.013200,0.152782
378,2024-04-05,La Salle-Italia,Autre ou hors France,Auriane,22.0,la-salle-italia,autre-ou-hors-france,NaN,NaN,NaN,NaN,NaN,45.745437,7.070565
423,2024-01-01,Bacouya,Nouvelle-Calédonie,Lucie,29.0,bacouya,nouvelle-caledonie,NaN,NaN,NaN,NaN,NaN,-21.562025,165.492567
445,2025-07-07,Coulanges-lès-Nevers,Nièvre,Simone,90.0,coulanges-les-nevers,nievre,NaN,NaN,NaN,NaN,NaN,47.005463,3.188076


In [39]:
df_merged_sans_nan = pd.concat([df_merged_sans_nan,df_merged_nan], ignore_index=True)
df_merged_sans_nan.sort_values(by='date', inplace=True)

In [40]:
df_merged_sans_nan

,date,commune,departement,prenom,age,nom_sans_accent,dep_sans_accent,dep_nom,dep_code,population,superficie_km2,densite,latitude,longitude
127,2022-01-01,Nice,Alpes-Maritimes,Lisa,45.0,nice,alpes-maritimes,Alpes-Maritimes,06,348085.0,74.0,4710.0,43.696000,7.272000
128,2022-01-01,Labry,Meurthe-et-Moselle,Muriel,56.0,labry,meurthe-et-moselle,Meurthe-et-Moselle,54,1575.0,6.0,254.0,49.173000,5.882000
129,2022-01-01,Bellevigne-les-Châteaux,Maine-et-Loire,Éléonore,27.0,bellevigne-les-chateaux,maine-et-loire,Maine-et-Loire,49,3463.0,7.0,527.0,47.214000,-0.075000
126,2022-01-08,Aubervilliers,Seine-Saint-Denis,Elsa-Marie,29.0,aubervilliers,seine-saint-denis,Seine-Saint-Denis,93,90071.0,6.0,15610.0,48.915000,2.382000
124,2022-01-14,Roujan,Hérault,Amélie,21.0,roujan,herault,Hérault,34,2260.0,17.0,133.0,43.504000,3.310000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,2025-08-13,11e arrondissement,Paris,Mme S,60.0,11e-arrondissement,paris,NaN,NaN,NaN,NaN,NaN,48.858416,2.379703
363,2025-08-15,Colombes,Hauts-de-Seine,–,NaN,colombes,hauts-de-seine,Hauts-de-Seine,92,88870.0,8.0,11423.0,48.922000,2.254000
365,2025-08-17,Petit Bourg,Guadeloupe,–,34.0,petit-bourg,guadeloupe,Guadeloupe,971,24272.0,131.0,185.0,16.192000,-61.590000
364,2025-08-17,Petit Bourg,Guadeloupe,–,17.0,petit-bourg,guadeloupe,Guadeloupe,971,24272.0,131.0,185.0,16.192000,-61.590000


In [ ]:
#sauvegarder df_merged_sans_nan dans un fichier csv
df_merged_sans_nan.to_csv('../data/processed/feminicide_2022_2025.csv', index=False)